In [17]:
# app.py

from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel, Field
import mlflow
import pandas as pd
import uvicorn
import logging

app = FastAPI()

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define your MLflow model name and version
model_name = "House-pricing-model"
model_version = "3"

# SQLite URI for MLflow tracking server
mlflow_uri = "sqlite:///mlflow.db"  # Replace with the actual path to your mlflow.db file

try:
    # Initialize MLflow client
    mlflow.set_tracking_uri(mlflow_uri)
    client = mlflow.tracking.MlflowClient()

    # Get registered model details
    model_details = client.get_registered_model(model_name)

    # Load the model using its latest version
    model_version_details = client.get_model_version(model_details.name, model_version)
    model_uri = model_version_details.source

    # Alternatively, you can load the model using specific version URI
    # model_uri = f"{mlflow_uri}#models:/{model_name}/{model_version}"

    # Load the model
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    logger.info(f"Successfully loaded MLflow model '{model_name}' version '{model_version}' from '{model_uri}'")
except Exception as e:
    logger.error(f"Failed to load MLflow model: {str(e)}")
    raise e

input = {
  "area": 1500,
  "bedrooms": 3,
  "bathrooms": 2,
  "stories": 2,
  "parking": 1,
  "furnished": 0,
  "semi_furnished": 1,
  "unfurnished": 0,
  "mainroad_new": 1
}
df = pd.DataFrame([input])

df.head()

prediction = loaded_model.predict(df)


INFO:__main__:Successfully loaded MLflow model 'House-pricing-model' version '3' from '/workspaces/MLOps_project/02-experiment_tracking/mlruns/1/4f4244ef75764938ad5bd8d08f8642ab/artifacts/model'


ValueError: feature_names mismatch: ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'furnished', 'semi-furnished', 'unfurnished', 'mainroad_new'] ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'furnished', 'semi_furnished', 'unfurnished', 'mainroad_new']
expected semi-furnished in input data
training data did not have the following fields: semi_furnished